In [1]:
# %% test cuda
import torch #type: ignore
torch.cuda.is_available()

True

In [2]:
# %% update data
import socket
from scripts.util.data.DataUpdater import update_server_main , update_laptop_main
from scripts.util.data.sqlConnector import update_sql_since , update_sql_dates

if socket.gethostname() == 'mengkjin-server':
    update_server_main()
else:
    update_laptop_main()
update_sql_since()
if False:
    update_sql_dates(20240101 , 20240229)

Use device name: NVIDIA GeForce RTX 4090
Merge Update Files
['/home/mengkjin/Workspace/SharedFolder/DB_updater.2.h5']
Thu Mar 14 22:24:13 2024 : DB_updater.2.h5 > data/DB_data/DB_information/DB_information.h5/./basic/calendar copying Done!
Thu Mar 14 22:24:22 2024 : DB_updater.2.h5 > data/DB_data/DB_information/DB_information.h5/./stock/concepts copying Done!
Thu Mar 14 22:24:22 2024 : DB_updater.2.h5 > data/DB_data/DB_information/DB_information.h5/./stock/description copying Done!
Thu Mar 14 22:24:39 2024 : DB_updater.2.h5 > data/DB_data/DB_information/DB_information.h5/./stock/industry copying Done!
Thu Mar 14 22:24:40 2024 : DB_updater.2.h5 > data/DB_data/DB_information/DB_information.h5/./stock/st copying Done!
Thu Mar 14 22:24:40 2024 : DB_updater.2.h5 > data/DB_data/DB_labels/DB_labels.2024.h5/./10days/lag0 copying Done!
Thu Mar 14 22:24:40 2024 : DB_updater.2.h5 > data/DB_data/DB_labels/DB_labels.2024.h5/./10days/lag1 copying Done!
Thu Mar 14 22:24:40 2024 : DB_updater.2.h5 > da

In [3]:
# %% test a specific model
from run_model import main
main(process = 0 , rawname = 1 , resume = 0 , anchoring = 0)

24-03-14 22:41:50|MOD:run_model   |: Model Specifics:
24-03-14 22:41:50|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train + Test + Instance
--Start Training New!
--Model_name is set to resnet_lstm_15m!
{'verbosity': 2,
 'storage_type': 'mem',
 'precision': 'float',
 'batch_size': 10000,
 'model_name': 'resnet_lstm_15m',
 'model_module': 'resnet_lstm',
 'model_data_type': '15m',
 'model_num': 3,
 'beg_date': 20170103,
 'end_date': 99991231,
 'interval': 120,
 'input_step_day': 5,
 'test_step_day': 1,
 'MODEL_PARAM': {'hidden_dim': [32],
                 'seqlens': [{'day': 40, '30m': 30, '15m': 20, 'dms': 40}],
                 'tra_seqlens': [{'hist_loss': 40}],
                 'dropout': [0.1],
                 'enc_in': [True],
                 'enc_att': [False],
                 'rnn_type': ['lstm'],
                 'rnn_att': [False],
                 'rnn_layers': [2],
                 'dec_mlp_layers': [1],
                 'num_output': [1],
                 'kernel_size': [3, 3],
                 'hidden_as_factor': [False],
                 'ordered_param_group':

24-03-14 22:43:47|MOD:run_model   |: Finish Process [Load Data]! Cost 117.3Secs
24-03-14 22:43:47|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [15m] : [endpoint_division(False) , history_standardize(True)]


resnet_lstm_15m #0 @20170103 LoadData Cost   18.9Secs


score function of [pearson] calculated and success!
loss function of [pearson] calculated and success!


FirstBite Ep#  0 : loss  1.00151, train-0.00144, valid-0.02315, max-0.0232, best-0.0232, lr1.0e-07
FirstBite Ep#  5 : loss  0.87619, train 0.13236, valid 0.10900, max 0.1221, best 0.1221, lr3.8e-03
FirstBite Ep# 10 : loss  0.85254, train 0.15975, valid 0.11617, max 0.1221, best 0.1221, lr1.3e-03
FirstBite Ep# 15 : loss  0.83279, train 0.18324, valid 0.11100, max 0.1221, best 0.1221, lr6.3e-04
FirstBite Ep# 20 : loss  0.82885, train 0.18800, valid 0.10187, max 0.1221, best 0.1221, lr1.3e-03
24-03-14 22:46:45|MOD:run_model   |: resnet_lstm_15m #0 @20170103|Round 0 FirstBite Ep# 23 EarlyStop|Train 0.1978 Valid 0.1054 BestVal 0.1221|Cost  3.0Min,  6.6Sec/Ep
FirstBite Ep#  0 : loss  0.99651, train 0.00354, valid-0.01823, max-0.0182, best-0.0182, lr1.0e-07
FirstBite Ep#  5 : loss  0.87908, train 0.12909, valid 0.12846, max 0.1285, best 0.1285, lr3.8e-03
FirstBite Ep# 10 : loss  0.85402, train 0.15798, valid 0.13431, max 0.1360, best 0.1360, lr1.3e-03
FirstBite Ep# 15 : loss  0.83410, train 0

In [5]:
import os
del_path = './instance/tra_lstm2_day_ShortTest'
if os.path.exists(del_path): os.rmdir(del_path)
# rm -r ./instance/tra_lstm2_day_ShortTest #type: ignore

In [ ]:
# %% model data preprocessing
from data_preprocessing import main
main(confirm=1)